### Movie Recommendation System (User-Based Collaborative Filtering)

- **Dataset**: MovieLens 100K (from Kaggle)
- **Task**: Recommend movies based on user similarity
- **Approach**:
  - Build a User-Item Matrix
  - Compute similarity scores between users
  - Recommend top-rated unseen movies
  - Evaluate with Precision@K

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score

In [4]:
# Load ratings file (userId, itemId, rating, timestamp)
ratings = pd.read_csv("u.data", sep="\t", names=["userId", "movieId", "rating", "timestamp"])

# Load movie info file
movies = pd.read_csv("u.item", sep="|", encoding="latin-1", 
                     names=["movieId", "title"] + [str(i) for i in range(22)], 
                     usecols=[0,1])

ratings.head()

,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [5]:
# Merge ratings with movie titles
data = pd.merge(ratings, movies, on="movieId")
data.head()

,userId,movieId,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,186,302,3,891717742,L.A. Confidential (1997)
2,22,377,1,878887116,Heavyweights (1994)
3,244,51,2,880606923,Legends of the Fall (1994)
4,166,346,1,886397596,Jackie Brown (1997)


In [6]:
# Create pivot table (users x movies)
user_item_matrix = data.pivot_table(index="userId", columns="title", values="rating")

user_item_matrix.fillna(0, inplace=True)
user_item_matrix.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,2.0,5.0,0.0,0.0,3.0,4.0,0.0,0.0,...,0.0,0.0,0.0,5.0,3.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,2.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0


In [7]:
# Compute cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix)

# Convert to DataFrame for better visualization
user_similarity_df = pd.DataFrame(user_similarity, 
                                  index=user_item_matrix.index, 
                                  columns=user_item_matrix.index)

user_similarity_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.168937,0.048388,0.064561,0.379670,0.429682,0.443097,0.320079,0.078385,0.377733,...,0.372213,0.119860,0.269860,0.193343,0.197949,0.118722,0.315064,0.149086,0.181612,0.399432
2,0.168937,1.000000,0.113393,0.179694,0.073623,0.242106,0.108604,0.104257,0.162470,0.161273,...,0.147095,0.310661,0.363328,0.410725,0.322713,0.231096,0.228793,0.162911,0.175273,0.106732
3,0.048388,0.113393,1.000000,0.349781,0.021592,0.074018,0.067423,0.084419,0.062039,0.066217,...,0.033885,0.043453,0.167140,0.071288,0.126278,0.026758,0.164539,0.102899,0.136757,0.026990
4,0.064561,0.179694,0.349781,1.000000,0.031804,0.068431,0.091507,0.188060,0.101284,0.060859,...,0.054615,0.036784,0.133619,0.196561,0.146058,0.030202,0.196858,0.152041,0.171538,0.058752
5,0.379670,0.073623,0.021592,0.031804,1.000000,0.238636,0.374733,0.248930,0.056847,0.201427,...,0.340183,0.080580,0.095284,0.081053,0.148607,0.071612,0.239955,0.139595,0.153799,0.313941


In [8]:
def recommend_movies(user_id, user_item_matrix, user_similarity_df, top_n=5):
    # Find similar users
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)
    similar_users = similar_users.drop(user_id)  # remove self
    
    # Take top 5 similar users
    top_users = similar_users.head(5).index
    
    # Movies rated by similar users
    similar_users_ratings = user_item_matrix.loc[top_users]
    
    # Average ratings
    avg_ratings = similar_users_ratings.mean().sort_values(ascending=False)
    
    # Exclude already watched movies
    watched = user_item_matrix.loc[user_id]
    recommendations = avg_ratings[watched == 0].head(top_n)
    
    return recommendations

In [9]:
# Example: Recommend for user 1
recommendations = recommend_movies(1, user_item_matrix, user_similarity_df, top_n=5)
print("Top Recommendations for User 1:\n")
print(recommendations)

Top Recommendations for User 1:

title
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)    4.4
Casablanca (1942)                                                              4.0
Stand by Me (1986)                                                             4.0
Heathers (1989)                                                                4.0
Piano, The (1993)                                                              3.8
dtype: float64


In [10]:
def precision_at_k(user_id, k=5):
    recommended = recommend_movies(user_id, user_item_matrix, user_similarity_df, top_n=k).index
    
    # actual liked movies (rating >= 4)
    actual = data[(data.userId == user_id) & (data.rating >= 4)]["title"].values
    
    hits = len(set(recommended) & set(actual))
    return hits / k

# Test for user 1
print("Precision@5 for User 1:", precision_at_k(1, k=5))

Precision@5 for User 1: 0.0
